In [3]:
from pathlib import Path
import sys
import numpy as np
import pandas as pd

# expose utils from terminal_voltage_calculation
project_root = Path("/home/kcv/Desktop/terminal_voltage_calculation")
sys.path.append(str(project_root))

from utils.ohmic_res_soc_helpers.cluster_cells import ResistanceClusterer
from utils.ohmic_res_soc_helpers.resistance_model import ResistanceModel

output_dir = Path("/home/kcv/Desktop/constant_power_test/post_processing/data/DRE_data")
output_dir.mkdir(parents=True, exist_ok=True)

# train on ohmic-resistance dataset
base_dir = Path("/home/kcv/Desktop/terminal_voltage_calculation/post_processed_data/ohmic_resistance_calculated")
clusterer = ResistanceClusterer(ohmic_output_dir=base_dir, n_clusters=3)
clusterer.fit()

res_model = ResistanceModel(ohmic_output_dir=base_dir, n_clusters=clusterer.n_clusters)
res_model.fit()

# predict for stitched detail data
predict_dir = Path("/home/kcv/Desktop/constant_power_test/stitched_detail_data")

def add_soc(df: pd.DataFrame) -> pd.DataFrame:
    df = df.copy()
    df["SOC"] = np.nan
    capacity = pd.to_numeric(df["Capacity(Ah)"], errors="coerce")

    for step_no, step_df in df.groupby("Step No"):
        idx = step_df.index
        step_cap = capacity.loc[idx].to_numpy(dtype=float)
        qmax = np.nanmax(np.abs(step_cap))
        if not np.isfinite(qmax) or qmax <= 0:
            continue

        name = str(step_df["Step name"].iloc[0]).lower()
        if "dchg" in name:
            soc = 100 * (qmax - np.abs(step_cap)) / qmax
        elif "chg" in name:
            soc = 100 * step_cap / qmax
        else:
            continue

        df.loc[idx, "SOC"] = soc

    return df

def predict_cluster_id(qchg_ah: float, qdchg_ah: float) -> int:
    return clusterer.predict_cluster(qchg_ah=qchg_ah, qdchg_ah=qdchg_ah)

def max_q_values(df: pd.DataFrame) -> tuple[float, float]:
    step = df["Step name"].astype(str)
    qchg = pd.to_numeric(
        df.loc[step.str.contains("Chg", case=False, na=False), "Capacity(Ah)"],
        errors="coerce",
    ).abs().max()
    qdchg = pd.to_numeric(
        df.loc[step.str.contains("CP_DChg", case=False, na=False), "Capacity(Ah)"],
        errors="coerce",
    ).abs().max()
    return float(qchg), float(qdchg)

processed = []
for csv_path in sorted(predict_dir.glob("*.csv")):
    df = pd.read_csv(csv_path)
    df = add_soc(df)

    qchg, qdchg = max_q_values(df)
    if not (np.isfinite(qchg) and np.isfinite(qdchg)):
        print(f"[skip] {csv_path.name}: missing Qmax")
        continue

    cluster_id = predict_cluster_id(qchg, qdchg)
    df["cluster_id"] = cluster_id
    df["Predicted_R_ohm"] = np.nan

    charge_mask = df["Step name"].astype(str).str.contains("Chg", case=False, na=False)
    discharge_mask = df["Step name"].astype(str).str.contains("DChg", case=False, na=False)

    if charge_mask.any():
        df.loc[charge_mask, "Predicted_R_ohm"] = res_model.predict_series(
            cluster_id, df.loc[charge_mask, "SOC"], step="charge"
        )

    if discharge_mask.any():
        df.loc[discharge_mask, "Predicted_R_ohm"] = res_model.predict_series(
            cluster_id, df.loc[discharge_mask, "SOC"], step="discharge"
        )

    out_path = output_dir / f"{csv_path.stem}_with_resistance.csv"
    df.to_csv(out_path, index=False)
    processed.append(out_path)
    print(f"[ok] {csv_path.name} → {out_path.name} (cluster {cluster_id})")

print(f"Done. Wrote {len(processed)} files to {output_dir}")


[ok] RD_LFP_ConstantPower_REPT_TS_150_0001_0_100.csv → RD_LFP_ConstantPower_REPT_TS_150_0001_0_100_with_resistance.csv (cluster 2)
[ok] RD_LFP_ConstantPower_REPT_TS_150_0003_0_100.csv → RD_LFP_ConstantPower_REPT_TS_150_0003_0_100_with_resistance.csv (cluster 2)
[ok] RD_LFP_ConstantPower_REPT_TS_150_0004_0_100.csv → RD_LFP_ConstantPower_REPT_TS_150_0004_0_100_with_resistance.csv (cluster 2)
[ok] RD_LFP_ConstantPower_REPT_TS_150_0007_0_100.csv → RD_LFP_ConstantPower_REPT_TS_150_0007_0_100_with_resistance.csv (cluster 2)
[ok] RD_LFP_ConstantPower_REPT_TS_150_0011_0_100.csv → RD_LFP_ConstantPower_REPT_TS_150_0011_0_100_with_resistance.csv (cluster 2)
[ok] RD_LFP_ConstantPower_REPT_TS_150_0012_0_100.csv → RD_LFP_ConstantPower_REPT_TS_150_0012_0_100_with_resistance.csv (cluster 2)
[ok] RD_LFP_ConstantPower_REPT_TS_150_0025_0_100.csv → RD_LFP_ConstantPower_REPT_TS_150_0025_0_100_with_resistance.csv (cluster 2)
[ok] RD_LFP_ConstantPower_REPT_TS_150_0034_0_100.csv → RD_LFP_ConstantPower_REPT_TS

In [9]:
from pathlib import Path

import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages
import pandas as pd

dre_dir = Path("/home/kcv/Desktop/constant_power_test/post_processing/data/DRE_data")
output_pdf = Path("/home/kcv/Desktop/constant_power_test/post_processing/plots/predicted_resistance_vs_soc.pdf")

frames = []
for csv_path in sorted(dre_dir.glob("*_with_resistance.csv")):
    df = pd.read_csv(csv_path)
    df = df[pd.notna(df["Predicted_R_ohm"]) & pd.notna(df["SOC"])]
    if df.empty:
        continue
    parts = csv_path.stem.replace("_with_resistance", "").split("_")
    cell_id = "_".join(parts[-3:]) if len(parts) >= 3 else parts[-1]
    df["Cell ID"] = cell_id
    frames.append(df)

if not frames:
    raise ValueError("No DRE files with predictions found.")

all_df = pd.concat(frames, ignore_index=True)

with PdfPages(output_pdf) as pdf:
    for (step_no, step_name), step_df in all_df.groupby(["Step No", "Step name"]):
        if step_df.empty:
            continue

        fig, ax = plt.subplots(figsize=(9, 5))
        for cell_id, cell_df in step_df.groupby("Cell ID"):
            cell_df = cell_df.sort_values("SOC")
            max_power_w = cell_df["Power(mW)"].abs().max() / 1000
            ax.plot(
                cell_df["SOC"],
                cell_df["Predicted_R_ohm"],
                marker="o",
                markersize=3,
                linewidth=1.1,
                label=f"{cell_id} — {max_power_w:.1f} W",
            )

        ax.set_title(f"{step_name} — Step {step_no} (Predicted R vs SOC)")
        ax.set_xlabel("SOC (%)")
        ax.set_ylabel("Predicted R (Ω)")
        ax.grid(alpha=0.3)
        ax.legend(title="Cell ID / Max |P|", bbox_to_anchor=(1.02, 1), loc="upper left")
        plt.tight_layout()
        pdf.savefig(fig)
        plt.close(fig)

print(f"Saved plots to {output_pdf}")


Saved plots to /home/kcv/Desktop/constant_power_test/post_processing/plots/predicted_resistance_vs_soc.pdf
